In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from torchvision.transforms import ToPILImage
import torch
img = Image.open("tmp/label/Abyssinian_1.png")
plt.imshow(img, cmap='gray')
plt.show()

width, height = img.size
pixels = img.load()

colors = [[pixels[w,h] for w in range(width)] for h in range(height)]

print(np.unique(colors))

# img = read_image("TrainVal/label/american_bulldog_10.png")
# print(torch.unique(img))
# img = ToPILImage()(img)
# pixels = img.load()

# colors = [[pixels[w,h] for w in range(width)] for h in range(height)]
# print(np.unique(colors))
# plt.imshow(img)
# plt.show()

# nimg = np.array(img)
# print(nimg)
# np.savetxt("test.txt", nimg, fmt='%.1f')

In [ ]:
from PIL import Image

image = "Abyssinian_1.jpg"
train = "TrainVal/color/"
target = "tmp/"

base_width = 512
img = Image.open(train+image)
wpercent = (base_width / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
img = img.resize((base_width, hsize), Image.Resampling.LANCZOS)
img.save(target+image)

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set the paths for the source and target directories
train_dir = "Test/color/"
target_dir = "rtest/color"
target_size = 512

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Retrieve list of image files with supported extensions
image_files = [f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

# Loop over all image files with a progress bar
for image_file in tqdm(image_files, desc="Processing images"):
    img_path = os.path.join(train_dir, image_file)
    img = Image.open(img_path)
    width, height = img.size

    # Calculate the new dimensions while maintaining aspect ratio
    if width > height:
        new_width = target_size
        new_height = int(height * (target_size / width))
    else:
        new_height = target_size
        new_width = int(width * (target_size / height))

    # Resize the image using high-quality resampling
    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Create a new blank image with a black background
    new_img = Image.new("RGB", (target_size, target_size), (0, 0, 0))
    
    # Calculate the position to paste the resized image
    paste_x = (target_size - new_width) // 2
    paste_y = (target_size - new_height) // 2

    # Paste the resized image onto the blank canvas
    new_img.paste(resized_img, (paste_x, paste_y))

    # Save the final image
    new_img.save(os.path.join(target_dir, image_file))

In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set the paths for the source and target directories
train_dir = "Test/label/"
target_dir = "rtest/label"
target_size = 512

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Retrieve list of image files with supported extensions
image_files = [f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

# Loop over all image files with a progress bar
for image_file in tqdm(image_files, desc="Processing images"):
    img_path = os.path.join(train_dir, image_file)
    
    # Open the image and convert to numpy array to ensure we can control the values
    img = Image.open(img_path)
    img_array = np.array(img)
    
    # Verify we have the expected values
    unique_values = np.unique(img_array)
    # print(f"Image {image_file} has unique values: {unique_values}")
    
    # Get dimensions
    height, width = img_array.shape[:2]
    
    # Calculate the new dimensions while maintaining aspect ratio
    if width > height:
        new_width = target_size
        new_height = int(height * (target_size / width))
    else:
        new_height = target_size
        new_width = int(width * (target_size / height))
    
    # Create a new blank array with zeros (black background)
    new_array = np.zeros((target_size, target_size), dtype=np.uint8)
    
    # Resize the original image using PIL with NEAREST resampling
    # This preserves the exact pixel values
    resized_img = img.resize((new_width, new_height), Image.Resampling.NEAREST)
    resized_array = np.array(resized_img)
    
    # Calculate the position to paste the resized image
    paste_y = (target_size - new_height) // 2
    paste_x = (target_size - new_width) // 2
    
    # Paste the resized image into the new array
    new_array[paste_y:paste_y+new_height, paste_x:paste_x+new_width] = resized_array
    
    # Verify the final array has only the expected values
    final_unique_values = np.unique(new_array)
    # print(f"Final image has unique values: {final_unique_values}")
    
    # Convert back to PIL Image and save
    new_img = Image.fromarray(new_array)
    new_img.save(os.path.join(target_dir, image_file))

In [ ]:
import os
import re

# Set the folder path containing the images (update this with your folder path)
folder_path = 'tmp/color'

# Compile the regex pattern:
# This matches file names ending with an underscore, one or more digits, and ".jpg".
pattern = re.compile(r'^.*_\d+\.jpg$', re.IGNORECASE)

# List to collect files that do NOT match the naming convention
non_matching_files = []

# Loop over all files in the specified folder
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    # Ensure we only check files
    if os.path.isfile(file_path):
        if not pattern.match(file):
            non_matching_files.append(file)

print("Files that do not follow the naming convention '<name>_<number>.jpg':")
print(non_matching_files)



In [ ]:
import os
import shutil

def move_jpg_files(source_folder, destination_folder):
    """
    Moves all JPG files from the source folder to the destination folder.

    Args:
        source_folder (str): The path to the source folder.
        destination_folder (str): The path to the destination folder.
    """

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder):
        # Check if the file ends with .jpg (case-insensitive)
        if filename.lower().endswith(".jpg"):
            # Create the full paths for the source and destination files
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(destination_folder, filename)

            try:
                # Move the file
                shutil.move(source_path, destination_path)
                print(f"Moved '{filename}' from '{source_folder}' to '{destination_folder}'")
            except Exception as e:
                print(f"Error moving '{filename}': {e}")

# Example usage (replace with your actual folder paths)
source_folder = "tmp"  # Replace with your source folder path
destination_folder = "tmp/color"  # Replace with your destination folder path

move_jpg_files(source_folder, destination_folder)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class down(nn.Module):
    def __init__(self, din, dout):
        super().__init__()
        self.down = nn.Sequential(
            nn.Conv2d(din, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
            nn.Conv2d(dout, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.down(x)
    
class up(nn.Module):
    def __init__(self, din, dout):
        super().__init__()
        self.up = nn.Sequential(
            nn.Conv2d(din, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
            nn.Conv2d(dout, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.up(x)

class unet(nn.Module):
    def __init__(self):
        super().__init__()
        self.scale = 3 # maybe using the original channels might be better
        self.flatten = nn.Flatten()
        self.down1 = down(3, self.scale * 64)
        self.down2 = down(self.scale * 64, self.scale * 128)
        self.down3 = down(self.scale * 128, self.scale * 256)
        self.down4 = down(self.scale * 256, self.scale * 512)
        self.down5 = down(self.scale * 512, self.scale * 1024)
        self.up1 = up(self.scale * 1024, self.scale * 512)
        self.up2 = up(self.scale * 512, self.scale * 256)
        self.up3 = up(self.scale * 256, self.scale * 128)
        self.up4 = up(self.scale * 128, self.scale * 64)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2)
        self.upconv1 = nn.Conv2d(self.scale * 1024, self.scale * 512, kernel_size=3, padding=1)
        self.upconv2 = nn.Conv2d(self.scale * 512, self.scale * 256, kernel_size=3, padding=1)
        self.upconv3 = nn.Conv2d(self.scale * 256, self.scale * 128, kernel_size=3, padding=1)
        self.upconv4 = nn.Conv2d(self.scale * 128, self.scale * 64, kernel_size=3, padding=1)
        self.output = nn.Sequential(
            nn.Conv2d(self.scale * 64, 4, kernel_size=1),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x1 = self.down1(x)
        x2 = self.down2(self.maxpool(x1))
        x3 = self.down3(self.maxpool(x2))
        x4 = self.down4(self.maxpool(x3))
        x5 = self.down5(self.maxpool(x4))
        x6 = self.up1(torch.cat([x4, self.upconv1(self.upsample(x5))], dim=1))
        x7 = self.up2(torch.cat([x3, self.upconv2(self.upsample(x6))], dim=1))
        x8 = self.up3(torch.cat([x2, self.upconv3(self.upsample(x7))], dim=1))
        x9 = self.up4(torch.cat([x1, self.upconv4(self.upsample(x8))], dim=1))
        pre_output = self.output(x9)
        return torch.argmax(pre_output, dim=1)



In [ ]:
# img = Image.open("tmp/color/Abyssinian_1.jpg")
# t = transforms.ToTensor()(img)
# t = t.unsqueeze(0)

# print(t.size())

model = unet()

with torch.no_grad():  # Disable gradient calculation for inference
    model.eval() # Set the model to evaluation mode
    res = model(t)
    print("Output tensor shape:", res.shape)
    print("Output tensor:\n",res)

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, PILToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


import os
from torchvision.io import read_image

class dataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_names = sorted([os.path.splitext(filename)[0] for filename in os.listdir(img_dir)])
        self.len = len(self.img_names)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        img = read_image(os.path.join(self.img_dir, self.img_names[idx] + ".jpg"))
        label = read_image(os.path.join(self.label_dir, self.img_names[idx] + ".png"))

        if self.transform:
            img = self.transform(img)

        if self.target_transform:
            label = self.target_transform(label)

        return img, label
    
def display_img_label(data, idx):
    img, label = data[idx]
    figure = plt.figure(figsize=(10,20))
    figure.add_subplot(1, 2, 1)
    plt.imshow(img.permute(1, 2, 0))

    figure.add_subplot(1, 2, 2)
    plt.imshow(label.permute(1, 2, 0), cmap='grey')

    plt.show()

class target_remap(object):
    def __call__(self, img):
        img[img == 255] = 3
        return img


training_data = dataset("rtrain/color", "rtrain/label", target_transform=target_remap())

test_data = dataset("rtest/color", "rtest/label", target_transform=target_remap())

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].permute(1, 2, 0)
# label = train_labels[0].permute(1, 2, 0)
# print(torch.unique(label))
# plt.imshow(img)
# plt.show()

# plt.imshow(label, cmap='grey')
# plt.show()

In [ ]:
sorted(os.listdir("tmp/color/"))

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(1,4,512, 512, requires_grad=True)
target = torch.empty(1,512, 512, dtype=torch.long).random_(3)
print(input.size())
print(target.size())
output = loss(input, target)
print(output)


In [ ]:
target[0,1] = 255

print(output)

In [ ]:
a = torch.log(torch.exp(input[0,0,0])/torch.sum(torch.exp(input[0,:,0])))
b = torch.log(torch.exp(input[0,1,1])/torch.sum(torch.exp(input[0,:,1])))
c = torch.log(torch.exp(input[0,0,2])/torch.sum(torch.exp(input[0,:,2])))
d = torch.log(torch.exp(input[0,0,3])/torch.sum(torch.exp(input[0,:,3])))
e = torch.log(torch.exp(input[0,1,4])/torch.sum(torch.exp(input[0,:,4])))
(a+b+c+d+e)/5

In [ ]:
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import decode_image

class dataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_names = sorted([os.path.splitext(filename)[0] for filename in os.listdir(img_dir)])
        self.len = len(self.img_names)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        img = decode_image(os.path.join(self.img_dir, self.img_names[idx] + ".jpg")).float()/255
        label = decode_image(os.path.join(self.label_dir, self.img_names[idx] + ".png"))

        if self.transform:
            img = self.transform(img)

        if self.target_transform:
            label = self.target_transform(label)

        return img, label
    
def display_img_label(data, idx):
    img, label = data[idx]
    figure = plt.figure(figsize=(10,20))
    figure.add_subplot(1, 2, 1)
    plt.imshow(img.permute(1, 2, 0))

    figure.add_subplot(1, 2, 2)
    plt.imshow(label.permute(1, 2, 0), cmap='grey')

    plt.show()

class target_remap(object):
    def __call__(self, img):
        img[img == 255] = 3
        return img

def diff_size_collate(batch):
    imgs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    return imgs, labels

target_batch_size = 64
batch_size = 1

training_data = dataset("rtrain/color", "rtrain/label", target_transform=target_remap())
val_data = dataset("Val/color", "Val/label", target_transform=target_remap())
test_data = dataset("Test/color", "Test/label", target_transform=target_remap())

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=diff_size_collate)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=diff_size_collate)

In [ ]:
import torch
from transformers import CLIPImageProcessor, CLIPVisionModel

# --- Configuration ---
MODEL_NAME = "openai/clip-vit-base-patch32" # Hugging Face model identifier

# --- 2. Setup Device and Load Model/Processor (Hugging Face) ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    # Load the specific vision model and the processor
    vision_model = CLIPVisionModel.from_pretrained(MODEL_NAME).to(device)
    processor = CLIPImageProcessor.from_pretrained(MODEL_NAME)
    vision_model.eval()
    print(f"Hugging Face CLIP Vision Model '{MODEL_NAME}' loaded successfully.")

except Exception as e:
    print(f"Error loading HF CLIP model: {e}")
    exit()


# --- 4. Get one image from the DataLoader and Perform Forward Pass ---
print("\nFetching one image batch from DataLoader...")

# Get the first batch (which contains one image in this case)

first_batch_pixel_values, _ = next(iter(train_dataloader))
# The DataLoader might return a list or tuple depending on the Dataset's __getitem__
# If __getitem__ returns only the tensor, the batch will be just the tensor batch
# Move the batch to the correct device
pixel_values_batch = first_batch_pixel_values.to(device)
print(f"Image batch loaded. Tensor shape: {pixel_values_batch.shape}")

pixel_values_batch = processor(images=pixel_values_batch, return_tensors="pt")["pixel_values"]
print(pixel_values_batch.shape)

print("\nPerforming forward pass using output_hidden_states=True...")
with torch.no_grad():
    outputs = vision_model(
        pixel_values=pixel_values_batch, # Use the batch from DataLoader
        output_hidden_states=True       # Request intermediate layer outputs
    )
print("Forward pass completed.")

# --- 5. Access and Print Hidden States ---
# outputs.hidden_states is a tuple.
hidden_states = outputs.hidden_states

print(f"\nCaptured {len(hidden_states)} hidden states for the image from the dataset:")
for i, state_tensor in enumerate(hidden_states):
    layer_desc = "Initial Embeddings" if i == 0 else f"Transformer Layer {i} Output"
    # Shape is (batch_size, sequence_length, hidden_size)
    print(f"  State {i} ({layer_desc}) Shape: {state_tensor.shape} (Device: {state_tensor.device})")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import CLIPVisionModel, CLIPVisionConfig, CLIPImageProcessor
from math import sqrt
import warnings

# --- Configuration ---
# Choose your CLIP model (adjust if using a different variant)
PRETRAINED_MODEL_NAME = "openai/clip-vit-base-patch32"
# Example image size (must be compatible with CLIP's expected input size)
# Standard CLIP ViT-B/16 takes 224x224
IMG_SIZE = 224
# Specify which transformer layers to use for skip connections
# Layer indices count from 0 (embeddings) up to num_hidden_layers
# Example: For ViT-Base (12 layers), use layers 3, 6, 9, and 12 (last)
# Note: hidden_states output includes initial embeddings as layer 0
# So, hidden_states[3] is output of layer 2, hidden_states[6] -> layer 5, etc.
# Let's target outputs *after* layers 2, 5, 8, 11 (indices 3, 6, 9, 12)
SKIP_LAYER_INDICES = [3, 6, 9, 12] # Example for ViT-B/16

# --- CLIP ViT Encoder ---
class ClipViTEncoder(nn.Module):
    def __init__(self, model_name=PRETRAINED_MODEL_NAME, freeze_encoder=True, skip_indices=SKIP_LAYER_INDICES):
        super().__init__()
        self.model_name = model_name
        self.skip_indices = sorted(skip_indices, reverse=True) # Get skips from deeper layers first

        # Load CLIP Vision Model configuration and weights
        self.config = CLIPVisionConfig.from_pretrained(model_name)
        self.clip_vit = CLIPVisionModel.from_pretrained(model_name)

        if freeze_encoder:
            for param in self.clip_vit.parameters():
                param.requires_grad = False

        # Calculate grid size (assuming square patches and square image)
        # Note: CLIP uses rectangular patches if image is not square, but
        # reshaping assumes square grid. Input should ideally be square.
        self.patch_size = self.config.patch_size
        self.grid_size = self.config.image_size // self.patch_size
        self.hidden_dim = self.config.hidden_size
        self.num_patches = self.grid_size * self.grid_size

        print(f"CLIP ViT Encoder initialized:")
        print(f"  Model: {model_name}")
        print(f"  Image Size: {self.config.image_size}x{self.config.image_size}")
        print(f"  Patch Size: {self.patch_size}x{self.patch_size}")
        print(f"  Grid Size: {self.grid_size}x{self.grid_size}")
        print(f"  Hidden Dim: {self.hidden_dim}")
        print(f"  Using skip connections from layer indices: {self.skip_indices}")
        print(f"  Encoder frozen: {freeze_encoder}")


    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input image tensor (B, C, H, W).
                               Make sure H, W match CLIP's expected size (e.g., 224x224)
                               and C=3. Preprocessing (normalization) should match CLIP's.
        Returns:
            tuple: (bottleneck_features, skip_features_list)
                   - bottleneck_features (torch.Tensor): Reshaped features from the final layer.
                                                        Shape: (B, hidden_dim, grid_size, grid_size)
                   - skip_features_list (list[torch.Tensor]): List of reshaped features
                                                             from specified intermediate layers.
                                                             Ordered from deepest to shallowest skip.
                                                             Each shape: (B, hidden_dim, grid_size, grid_size)
        """
        if x.shape[2] != self.config.image_size or x.shape[3] != self.config.image_size:
             warnings.warn(
                 f"Input image size ({x.shape[2]}x{x.shape[3]}) doesn't match "
                 f"CLIP expected size ({self.config.image_size}x{self.config.image_size}). "
                 f"Behavior may be unexpected. Consider resizing input."
             )

        # Pass image through CLIP ViT
        # output_hidden_states=True is crucial
        outputs = self.clip_vit(pixel_values=x, output_hidden_states=True)

        # `hidden_states` is a tuple: (embeddings, hidden_state_layer_1, ..., hidden_state_layer_N)
        # Length is num_hidden_layers + 1
        all_hidden_states = outputs.hidden_states

        # --- Extract and Reshape Features ---
        skip_features_list = []
        bottleneck_features = None

        # We iterate through specified layers to extract skips
        # The deepest specified layer serves as the bottleneck input for the decoder
        is_bottleneck_set = False
        for i in self.skip_indices:
            hidden_state = all_hidden_states[i] # Shape: (B, num_patches + 1, hidden_dim)

            # Remove the CLS token embedding (first token)
            # Shape -> (B, num_patches, hidden_dim)
            patch_embeddings = hidden_state[:, 1:, :]

            # Check if num_patches matches calculated grid size squared
            if patch_embeddings.shape[1] != self.num_patches:
                 # This might happen if image size is not what model expected,
                 # or if non-square patches/images lead to different seq lengths.
                 current_num_patches = patch_embeddings.shape[1]
                 current_grid_size_h = int(sqrt(current_num_patches)) # Approx calc
                 current_grid_size_w = current_num_patches // current_grid_size_h
                 if current_grid_size_h * current_grid_size_w != current_num_patches:
                     raise ValueError(f"Cannot reliably reshape patch embeddings. "
                                      f"Expected {self.num_patches} patches, got {current_num_patches}.")
                 # print(f"Warning: Actual patch count {current_num_patches} differs from expected {self.num_patches}. Trying to reshape to {current_grid_size_h}x{current_grid_size_w}.")
                 # Reshape to (B, grid_h, grid_w, C)
                 reshaped_features = patch_embeddings.reshape(
                     x.shape[0], current_grid_size_h, current_grid_size_w, self.hidden_dim
                 )
                 # Permute to (B, C, grid_h, grid_w) - PyTorch format
                 reshaped_features = reshaped_features.permute(0, 3, 1, 2).contiguous()

            else:
                 # Reshape to (B, grid_size, grid_size, C)
                 reshaped_features = patch_embeddings.reshape(
                     x.shape[0], self.grid_size, self.grid_size, self.hidden_dim
                 )
                 # Permute to (B, C, grid_size, grid_size) - PyTorch format
                 reshaped_features = reshaped_features.permute(0, 3, 1, 2).contiguous()


            if not is_bottleneck_set:
                bottleneck_features = reshaped_features
                is_bottleneck_set = True
            else:
                 # Add to skips (these come from shallower layers than bottleneck)
                skip_features_list.append(reshaped_features)

        # The skip list is currently deepest -> shallowest. Reverse it?
        # No, typically the decoder expects skips from shallow -> deep encoder layers
        # But our indices were sorted reverse, so bottleneck = deepest, list = next deeper -> shallowest
        # Standard U-Net decoder takes skips shallowest first. Let's keep list order as is (deep->shallow)
        # and the decoder can pop() or index accordingly. Or reverse here.
        # Let's reverse to match typical U-Net expectation (shallowest first in list)
        skip_features_list.reverse()

        if bottleneck_features is None:
             raise ValueError("Could not extract bottleneck features. Check skip_indices.")

        return bottleneck_features, skip_features_list

# --- Example U-Net Decoder Block ---
# This needs to be adapted based on the SKIP connection properties
class DecoderBlock(nn.Module):
    def __init__(self, in_channels_upsample, in_channels_skip, out_channels, skip_spatial_size, target_spatial_size):
        super().__init__()
        self.skip_spatial_size = skip_spatial_size
        self.target_spatial_size = target_spatial_size

        # Upsample the input from the previous decoder layer
        # Using ConvTranspose2d, factor should align target_spatial_size / prev_size
        # Or use nn.Upsample + Conv2d
        # Assuming stride 2 for doubling resolution typically
        self.upsample = nn.ConvTranspose2d(in_channels_upsample, in_channels_upsample // 2, kernel_size=2, stride=2)
        # Adjust number of channels in the skip connection if needed
        # All our ViT skips have hidden_dim channels and grid_size x grid_size spatial dim
        self.skip_conv = nn.Conv2d(in_channels_skip, in_channels_upsample // 2, kernel_size=1) # 1x1 conv
        # Double convolution block after concatenation
        conv_in_channels = in_channels_upsample // 2 + in_channels_upsample // 2 # After concat
        self.conv_block = nn.Sequential(
            nn.Conv2d(conv_in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x_upsample, x_skip):
        """
        Args:
            x_upsample: Feature map from the previous (deeper) decoder layer
            x_skip: Feature map (skip connection) from the corresponding encoder layer
        """
        x_upsample = self.upsample(x_upsample) # Upsample (e.g., H/2, W/2 -> H, W)

        # Adapt skip connection channels
        x_skip = self.skip_conv(x_skip) # Projects skip channels

        # --- Spatial Dimension Handling (CRITICAL) ---
        # The ViT skips might all be grid_size x grid_size.
        # The upsampled features x_upsample will have increasing resolution.
        # We need to ensure they match before concatenation.

        # Option 1: Resize skip connection to match x_upsample (may lose info)
        if x_skip.shape[2:] != x_upsample.shape[2:]:
              x_skip = F.interpolate(x_skip, size=x_upsample.shape[2:], mode='bilinear', align_corners=False)
              # print(f"Resized skip connection from {self.skip_spatial_size}x{self.skip_spatial_size} to {x_upsample.shape[2:]}")


        # Option 2: Design decoder so x_upsample always matches skip size (less standard U-Net)


        # Concatenate along the channel dimension
        x = torch.cat([x_upsample, x_skip], dim=1)

        # Apply convolutions
        x = self.conv_block(x)
        return x


# --- Example U-Net Decoder ---
# This needs careful design based on the number of skips and their properties
class UNetDecoder(nn.Module):
    def __init__(self, encoder_hidden_dim, encoder_grid_size, decoder_channels):
        super().__init__()
        self.encoder_hidden_dim = encoder_hidden_dim  # Channels in ViT features
        self.encoder_grid_size = encoder_grid_size   # Spatial size of ViT features (H/P, W/P)
        self.decoder_channels = decoder_channels     # e.g., [512, 256, 128, 64] - channels *after* each block

        # Example: 4 decoder blocks corresponding to 4 skip connections (+ bottleneck)
        num_blocks = len(decoder_channels)
        if num_blocks != len(SKIP_LAYER_INDICES) -1: # Need one less block than total skips extracted if one is bottleneck
            warnings.warn(f"Number of decoder blocks ({num_blocks}) "
                           f"doesn't match number of skip connection layers ({len(SKIP_LAYER_INDICES) -1}). Adjust config.")
            # Adjusting num_blocks based on provided decoder_channels
            num_blocks = len(decoder_channels)


        # Determine input channels for each block
        # Bottleneck has encoder_hidden_dim channels
        # First block takes bottleneck (upsampled) + deepest skip (projected)
        # Subsequent blocks take previous block output (upsampled) + next skip (projected)
        in_channels_upsample = encoder_hidden_dim # From bottleneck initially
        block_input_sizes = [] # To track expected spatial sizes

        self.blocks = nn.ModuleList()
        current_spatial_size = encoder_grid_size
        for i in range(num_blocks):
            out_ch = decoder_channels[i]
            # All ViT skips have encoder_hidden_dim channels and encoder_grid_size spatial size
            in_channels_skip = encoder_hidden_dim
            skip_spatial_size = encoder_grid_size

            # Target size after upsampling (typically doubles)
            target_spatial_size = current_spatial_size * 2 # Assuming stride-2 upsampling

            block = DecoderBlock(
                in_channels_upsample=in_channels_upsample,
                in_channels_skip=in_channels_skip,
                out_channels=out_ch,
                skip_spatial_size=skip_spatial_size,
                target_spatial_size=target_spatial_size
            )
            self.blocks.append(block)

            # Input channels for the *next* block's upsample path is the output of this block
            in_channels_upsample = out_ch
            current_spatial_size = target_spatial_size # Update expected spatial size

    def forward(self, bottleneck_features, skip_features):
        """
        Args:
            bottleneck_features (torch.Tensor): Output from the deepest ViT layer used.
                                                Shape: (B, encoder_hidden_dim, G, G)
            skip_features (list[torch.Tensor]): List of skip features from ViT,
                                                ordered shallowest layer first.
                                                Each: (B, encoder_hidden_dim, G, G)
        Returns:
            torch.Tensor: Output feature map from the final decoder block.
        """
        x = bottleneck_features
        # skip_features list is ordered shallowest -> deepest
        # We need to pair decoder blocks (deep -> shallow) with skips (deep -> shallow)
        # Let's assume skip_features were reversed by encoder: shallowest first in list
        # Decoder blocks run from deep -> shallow output
        for i, block in enumerate(self.blocks):
            # Get the corresponding skip connection
            # If list is shallow->deep, i=0 block (deepest) uses skip_features[-1] (deepest skip)
            # Let's reverse the list index access
            skip = skip_features[len(skip_features) - 1 - i]
            x = block(x, skip) # Pass current features and corresponding skip
        return x

# --- Combined CLIP-U-Net Model ---
class ClipUNet(nn.Module):
    def __init__(self, num_classes=4, decoder_channels=[512, 256, 128, 64], freeze_encoder=True):
        super().__init__()

        self.encoder = ClipViTEncoder(freeze_encoder=freeze_encoder)

        self.decoder = UNetDecoder(
            encoder_hidden_dim=self.encoder.hidden_dim,
            encoder_grid_size=self.encoder.grid_size,
            decoder_channels=decoder_channels
        )

        # Final 1x1 convolution to map to the number of classes
        self.final_conv = nn.Conv2d(decoder_channels[-1], num_classes, kernel_size=1)

        # Potentially add a final upsampling layer if the last decoder block's output
        # resolution doesn't match the original image size.
        # The last decoder block outputs at encoder_grid_size * 2^num_blocks
        final_decoder_size = self.encoder.grid_size * (2**len(decoder_channels))
        if final_decoder_size != IMG_SIZE:
             print(f"Final decoder size {final_decoder_size} != Input Image Size {IMG_SIZE}. Adding final upsample.")
             self.final_upsample = nn.Upsample(size=(IMG_SIZE, IMG_SIZE), mode='bilinear', align_corners=False)
        else:
             self.final_upsample = nn.Identity()


    def forward(self, x):
        # Ensure input is preprocessed correctly for CLIP!
        # (Resize to 224x224, normalize with CLIP's mean/std)
        
        bottleneck, skips = self.encoder(x)
        decoder_output = self.decoder(bottleneck, skips)
        
        # Final processing
        output = self.final_conv(decoder_output)
        output = self.final_upsample(output) # Upsample to original image size if needed

        return output



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = CLIPImageProcessor.from_pretrained(PRETRAINED_MODEL_NAME)

images, labels = next(iter(train_dataloader))

batch = processor(images=images, return_tensors="pt")["pixel_values"]

# Instantiate the model
model = ClipUNet(num_classes=4, decoder_channels=[512, 256, 128, 64]).to(device) # Example for 4 skips


# Forward pass
with torch.no_grad(): # Use no_grad if just inferencing or checking shapes
        output = model(batch)

print(f"\nInput shape: {batch.shape}")
print(f"Output shape: {output.shape}") # Should be (B, num_classes, H, W) e.g., (2, 5, 224, 224)

# --- Example: Manually check encoder outputs ---
print("\n--- Manual Encoder Check ---")
encoder = ClipViTEncoder().to(device)
bottleneck, skips = encoder(batch)
print(f"Bottleneck shape: {bottleneck.shape}") # e.g., (2, 768, 14, 14) for ViT-B/16
print(f"Number of skip features: {len(skips)}")
for i, skip in enumerate(skips):
    print(f"  Skip {i} (from shallowest layer used) shape: {skip.shape}") # e.g., (2, 768, 14, 14)
print("-" * 20)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import CLIPVisionModel, CLIPVisionConfig, CLIPImageProcessor
from math import sqrt
import warnings

# --- Configuration ---
# Choose your CLIP model (adjust if using a different variant)
PRETRAINED_MODEL_NAME = "openai/clip-vit-base-patch16" # CHANGED to patch32
# Example image size (must be compatible with CLIP's expected input size)
IMG_SIZE = 224 # Standard for CLIP
# Specify which transformer layers to use for skip connections
# Using 4 layers: Deepest (12) is bottleneck, others (3, 6, 9) are skips
SKIP_LAYER_INDICES = [3, 5, 7, 9, 12]

# --- CLIP ViT Encoder ---
class ClipViTEncoder(nn.Module):
    def __init__(self, model_name=PRETRAINED_MODEL_NAME, freeze_encoder=True, skip_indices=SKIP_LAYER_INDICES):
        super().__init__()
        # print("\n--- Initializing ClipViTEncoder ---")
        self.model_name = model_name
        self.skip_indices = sorted(skip_indices, reverse=True) # Process deeper layers first for bottleneck logic

        # Load CLIP Vision Model configuration and weights
        self.config = CLIPVisionConfig.from_pretrained(model_name)
        self.clip_vit = CLIPVisionModel.from_pretrained(model_name)

        if freeze_encoder:
            for param in self.clip_vit.parameters():
                param.requires_grad = False

        # Calculate grid size
        self.patch_size = self.config.patch_size
        # Ensure image size in config matches expected if not default
        if self.config.image_size != IMG_SIZE:
             warnings.warn(f"Model config image size {self.config.image_size} differs from specified IMG_SIZE {IMG_SIZE}. Using model config size for grid calculation.")
        self.grid_size = self.config.image_size // self.patch_size
        self.hidden_dim = self.config.hidden_size
        self.num_patches = self.grid_size * self.grid_size

        # print(f"  Model: {model_name}")
        # print(f"  Expected Input Image Size (from config): {self.config.image_size}x{self.config.image_size}")
        # print(f"  Patch Size: {self.patch_size}x{self.patch_size}")
        # print(f"  Calculated Grid Size (G): {self.grid_size}x{self.grid_size}")
        # print(f"  Calculated Num Patches (N_p = G*G): {self.num_patches}")
        # print(f"  Hidden Dim (D_vit): {self.hidden_dim}")
        # print(f"  Targeting hidden states at indices: {self.skip_indices}")
        # print(f"  Encoder frozen: {freeze_encoder}")
        # print("--- ClipViTEncoder Initialized ---\n")


    def forward(self, x):
        # print("\n--- ClipViTEncoder Forward ---")
        # print(f"  Input shape: {x.shape}")
        if x.shape[2] != self.config.image_size or x.shape[3] != self.config.image_size:
             warnings.warn(
                 f"Input image size ({x.shape[2]}x{x.shape[3]}) doesn't match "
                 f"CLIP expected size ({self.config.image_size}x{self.config.image_size}). "
                 f"Behavior may be unexpected. Consider resizing input."
             )

        # Pass image through CLIP ViT
        outputs = self.clip_vit(pixel_values=x, output_hidden_states=True)
        all_hidden_states = outputs.hidden_states
        # print(f"  Number of hidden states outputs (incl. embeddings): {len(all_hidden_states)}")
        # if all_hidden_states:
            # print(f"  Shape of one hidden state (e.g., index 0 - embeddings): {all_hidden_states[0].shape}") # e.g., (B, N_p+1, D_vit)

        # --- Extract and Reshape Features ---
        skip_features_list = []
        bottleneck_features = None
        is_bottleneck_set = False

        # print("  Extracting features from specified hidden state indices:")
        for i in self.skip_indices: # Iterates [12, 9, 6, 3]
            # print(f"    Processing hidden state index: {i}")
            hidden_state = all_hidden_states[i]
            # print(f"      Original hidden_state shape: {hidden_state.shape}") # Should be (B, N_p+1, D_vit)

            # Remove the CLS token embedding
            patch_embeddings = hidden_state[:, 1:, :]
            # print(f"      Shape after removing CLS token: {patch_embeddings.shape}") # Should be (B, N_p, D_vit)

            # Check num_patches consistency
            if patch_embeddings.shape[1] != self.num_patches:
                 # Handle potential mismatch (e.g., if input size wasn't exactly config.image_size)
                 current_num_patches = patch_embeddings.shape[1]
                 current_grid_size = int(sqrt(current_num_patches))
                 if current_grid_size * current_grid_size != current_num_patches:
                     raise ValueError(f"Cannot reliably reshape patch embeddings. Non-square grid? Expected {self.num_patches} patches, got {current_num_patches}.")
                 warnings.warn(f"Actual patch count {current_num_patches} differs from expected {self.num_patches}. Reshaping to {current_grid_size}x{current_grid_size}.")
                 grid_h, grid_w = current_grid_size, current_grid_size
            else:
                 grid_h, grid_w = self.grid_size, self.grid_size

            # Reshape to (B, grid_h, grid_w, D_vit) -> (B, D_vit, grid_h, grid_w)
            reshaped_features = patch_embeddings.reshape(
                x.shape[0], grid_h, grid_w, self.hidden_dim
            )
            reshaped_features = reshaped_features.permute(0, 3, 1, 2).contiguous()
            # print(f"      Shape after reshaping to grid (B, D_vit, G, G): {reshaped_features.shape}")

            if not is_bottleneck_set:
                # print(f"      --> Assigning to bottleneck_features")
                bottleneck_features = reshaped_features
                is_bottleneck_set = True
            else:
                # print(f"      --> Appending to intermediate skip list")
                skip_features_list.append(reshaped_features)

        # Reverse the collected skips to be shallowest feature first
        skip_features_list.reverse() # Now order matches [index_3_out, index_6_out, index_9_out]
        # print(f"  Finished extracting features.")
        # print(f"    Bottleneck features final shape: {bottleneck_features.shape if bottleneck_features is not None else 'None'}")
        # print(f"    Number of skip features collected: {len(skip_features_list)}")
        # for idx, skip in enumerate(skip_features_list):
            # print(f"      Skip {idx} (shallowest first) final shape: {skip.shape}")

        if bottleneck_features is None:
             raise ValueError("Could not extract bottleneck features. Check skip_indices.")

        # print("--- ClipViTEncoder Forward End ---")
        return bottleneck_features, skip_features_list

# --- Example U-Net Decoder Block ---
class DecoderBlock(nn.Module):
    def __init__(self, block_index, in_channels_upsample, in_channels_skip, out_channels, skip_spatial_size, target_spatial_size):
        super().__init__()
        self.block_index = block_index
        # print(f"    Initializing DecoderBlock {self.block_index}: Upsample_in={in_channels_upsample}, Skip_in={in_channels_skip}, Out={out_channels}")
        self.skip_spatial_size = skip_spatial_size
        self.target_spatial_size = target_spatial_size
        self.upsample_out_channels = in_channels_upsample // 2
        self.skip_conv_out_channels = in_channels_upsample // 2 # Match for concatenation

        self.upsample = nn.ConvTranspose2d(in_channels_upsample, self.upsample_out_channels, kernel_size=2, stride=2)
        self.skip_conv = nn.Conv2d(in_channels_skip, self.skip_conv_out_channels, kernel_size=1) # 1x1 conv

        conv_in_channels = self.upsample_out_channels + self.skip_conv_out_channels
        # print(f"      Block {self.block_index}: Upsample out channels={self.upsample_out_channels}, SkipConv out channels={self.skip_conv_out_channels}, Concat channels={conv_in_channels}")
        self.conv_block = nn.Sequential(
            nn.Conv2d(conv_in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x_upsample, x_skip):
        # print(f"\n  --- DecoderBlock {self.block_index} Forward ---")
        # print(f"    Input x_upsample shape: {x_upsample.shape}")
        # print(f"    Input x_skip shape: {x_skip.shape}")

        x_upsample = self.upsample(x_upsample)
        # print(f"    Shape after upsample: {x_upsample.shape}")

        x_skip_proj = self.skip_conv(x_skip)
        # print(f"    Shape after skip_conv (channel adjust): {x_skip_proj.shape}")

        # Resize skip connection spatially if needed
        if x_skip_proj.shape[2:] != x_upsample.shape[2:]:
            #   print(f"    Resizing skip connection from {x_skip_proj.shape[2:]} to {x_upsample.shape[2:]}")
              x_skip_resized = F.interpolate(x_skip_proj, size=x_upsample.shape[2:], mode='bilinear', align_corners=False)
            #   print(f"    Shape after skip resize: {x_skip_resized.shape}")
        else:
              x_skip_resized = x_skip_proj
            #   print(f"    Skip connection spatial size matches upsample, no resize needed.")

        # Concatenate
        x = torch.cat([x_upsample, x_skip_resized], dim=1)
        # print(f"    Shape after concatenation: {x.shape}")

        # Apply convolutions
        x = self.conv_block(x)
        # print(f"    Shape after conv_block (Output): {x.shape}")
        # print(f"  --- DecoderBlock {self.block_index} Forward End ---")
        return x

# --- Example U-Net Decoder ---
class UNetDecoder(nn.Module):
    def __init__(self, encoder_hidden_dim, encoder_grid_size, decoder_channels):
        super().__init__()
        # print("\n--- Initializing UNetDecoder ---")
        self.encoder_hidden_dim = encoder_hidden_dim
        self.encoder_grid_size = encoder_grid_size
        self.decoder_channels = decoder_channels
        num_blocks = len(decoder_channels)

        # *** Important Check ***
        num_expected_skips = len(SKIP_LAYER_INDICES) - 1 # One layer is bottleneck
        # print(f"  Encoder provides {num_expected_skips} skip connections (indices besides bottleneck).")
        # print(f"  Decoder configured with {num_blocks} blocks for channels: {decoder_channels}.")
        if num_blocks != num_expected_skips:
            warnings.warn(
               f"*** POTENTIAL MISMATCH: Number of decoder blocks ({num_blocks}) "
               f"does not match the number of available skip connections ({num_expected_skips}). "
               f"This might lead to an IndexError during forward pass if not handled carefully. ***"
            )
        # The code proceeds to create num_blocks based on decoder_channels length

        in_channels_upsample = encoder_hidden_dim
        self.blocks = nn.ModuleList()
        current_spatial_size = encoder_grid_size
        # print(f"  Creating {num_blocks} DecoderBlock(s):")
        for i in range(num_blocks):
            out_ch = decoder_channels[i]
            in_channels_skip = encoder_hidden_dim
            skip_spatial_size = encoder_grid_size
            target_spatial_size = current_spatial_size * 2 # Assumes stride-2 upsampling

            # Pass index for clearer prints inside block
            block = DecoderBlock(
                block_index=i,
                in_channels_upsample=in_channels_upsample,
                in_channels_skip=in_channels_skip,
                out_channels=out_ch,
                skip_spatial_size=skip_spatial_size,
                target_spatial_size=target_spatial_size
            )
            self.blocks.append(block)

            in_channels_upsample = out_ch # Next block's upsample input is this block's output
            current_spatial_size = target_spatial_size

        # print("--- UNetDecoder Initialized ---\n")


    def forward(self, bottleneck_features, skip_features):
        # print("\n--- UNetDecoder Forward ---")
        # print(f"  Input bottleneck_features shape: {bottleneck_features.shape}")
        # print(f"  Input skip_features list length: {len(skip_features)}")
        x = bottleneck_features
        num_available_skips = len(skip_features)

        # Iterate through decoder blocks (indices 0 to num_blocks-1)
        for i, block in enumerate(self.blocks):
            # print(f"  Processing Decoder Block {i}")
            # Calculate index for skip_features list (shallowest first)
            # Block 0 (deepest decoder) uses last skip (deepest skip feature)
            skip_idx = num_available_skips - 1 - i
            # print(f"    Attempting to use skip feature at index: {skip_idx} (from list of {num_available_skips})")

            if skip_idx < 0 or skip_idx >= num_available_skips:
                 # This will happen if num_blocks > num_available_skips
                 raise IndexError(f"Attempted to access skip_features[{skip_idx}] but only {num_available_skips} skips are available. Mismatch between decoder blocks and encoder skips.")

            skip = skip_features[skip_idx]
            # print(f"    Using skip feature with shape: {skip.shape}")
            x = block(x, skip) # Pass current features and corresponding skip

        # print(f"  Final output shape from UNetDecoder: {x.shape}")
        # print("--- UNetDecoder Forward End ---")
        return x

# --- Combined CLIP-U-Net Model ---
class ClipUNet(nn.Module):
    def __init__(self, num_classes=4, decoder_channels=[512, 256, 128, 64], freeze_encoder=True):
        super().__init__()
        # print("\n--- Initializing ClipUNet ---")
        # print(f"  Num output classes: {num_classes}")
        # print(f"  Decoder channels specified: {decoder_channels}")

        self.encoder = ClipViTEncoder(freeze_encoder=freeze_encoder)

        self.decoder = UNetDecoder(
            encoder_hidden_dim=self.encoder.hidden_dim,
            encoder_grid_size=self.encoder.grid_size,
            decoder_channels=decoder_channels
        )

        # Final 1x1 convolution
        last_decoder_channel = decoder_channels[-1] if decoder_channels else self.encoder.hidden_dim # Handle no decoder case?
        # print(f"  Final Conv: {last_decoder_channel} -> {num_classes} channels")
        self.final_conv = nn.Conv2d(last_decoder_channel, num_classes, kernel_size=1)

        # Final upsampling layer check
        final_decoder_size = self.encoder.grid_size * (2**len(decoder_channels))
        # print(f"  Calculated final decoder spatial size: {final_decoder_size}x{final_decoder_size}")
        if final_decoder_size != IMG_SIZE:
            #  print(f"  Final decoder size {final_decoder_size} != Target Image Size {IMG_SIZE}. Adding final Upsample layer.")
             self.final_upsample = nn.Upsample(size=(IMG_SIZE, IMG_SIZE), mode='bilinear', align_corners=False)
        else:
            #  print(f"  Final decoder size matches target image size. Final upsample is Identity.")
             self.final_upsample = nn.Identity()

        # print("--- ClipUNet Initialized ---\n")


    def forward(self, x):
        # print("\n\n========== ClipUNet Forward Pass Start ==========")
        # print(f"Overall Input shape: {x.shape}")

        # 1. Encoder
        bottleneck, skips = self.encoder(x)
        # print("\n--- Back in ClipUNet Forward (after Encoder) ---")
        # print(f"  Encoder returned bottleneck shape: {bottleneck.shape}")
        # print(f"  Encoder returned {len(skips)} skip features.")

        # 2. Decoder
        decoder_output = self.decoder(bottleneck, skips)
        # print("\n--- Back in ClipUNet Forward (after Decoder) ---")
        # print(f"  Decoder returned output shape: {decoder_output.shape}")

        # 3. Final Convolution
        output = self.final_conv(decoder_output)
        # print("\n--- Back in ClipUNet Forward (after Final Conv) ---")
        # print(f"  Shape after final_conv: {output.shape}")

        # 4. Final Upsampling (if needed)
        output = self.final_upsample(output)
        # print("\n--- Back in ClipUNet Forward (after Final Upsample) ---")
        # print(f"  Shape after final_upsample: {output.shape}")

        # print("========== ClipUNet Forward Pass End ==========\n")
        return output



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configuration for the run
BATCH_SIZE = 2 # Smaller batch size for testing
NUM_CLASSES = 4
# *** NOTE: This configuration will likely cause an error! ***
# We request 4 decoder blocks ([512, 256, 128, 64])
# But provide only 3 skips (from layers 3, 6, 9, with 12 as bottleneck)
DECODER_CHANNELS_CONFIG = [512, 256, 128, 64]

print("\n--- Preparing Example Data ---")
# Use CLIPProcessor to get correct preprocessing
try:
    processor = CLIPImageProcessor.from_pretrained(PRETRAINED_MODEL_NAME)
    # Create dummy images (replace with your actual data loading)
    # Generating random images often works, but using processor is better practice
    images, labels = next(iter(train_dataloader))
    # Process images
    batch_processed = processor(images=images, return_tensors="pt")["pixel_values"].to(device)
    print(f"  Processed batch shape: {batch_processed.shape}")
except Exception as e:
    print(f"  Error loading processor or processing dummy images: {e}")
    print(f"  Falling back to simple random tensor.")
    # Fallback simple tensor
    batch_processed = torch.randn(BATCH_SIZE, 3, IMG_SIZE, IMG_SIZE).to(device)
    print(f"  Using dummy tensor input shape: {batch_processed.shape}")


print("\n--- Instantiating Model ---")
# Instantiate the model
model = ClipUNet(num_classes=NUM_CLASSES,
                    decoder_channels=DECODER_CHANNELS_CONFIG, # Using the potentially problematic config
                    freeze_encoder=True).to(device)

print(sum(p.numel() for p in model.parameters() if p.requires_grad))

print("\n--- Running Forward Pass ---")
try:
    # Forward pass
    model.eval() # Set to evaluation mode
    with torch.no_grad(): # Use no_grad for inference example
        output = model(batch_processed)

    print(f"\n--- Forward Pass Successful ---")
    print(f"Final Output shape: {output.shape}") # Should be (B, num_classes, H, W)
    # Expected: (2, 4, 224, 224) if no error occurs
except IndexError as e:
        print(f"\n--- EXPECTED INDEX ERROR OCCURRED ---")
        print(f"  Error message: {e}")
        print(f"  This likely happened because the number of decoder blocks ({len(DECODER_CHANNELS_CONFIG)})")
        print(f"  is greater than the number of available skip connections ({len(SKIP_LAYER_INDICES) - 1}).")
except Exception as e:
    print(f"\n--- UNEXPECTED ERROR DURING FORWARD PASS ---")
    print(f"  Error type: {type(e)}")
    print(f"  Error message: {e}")


